In [4]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import log_loss, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier



In [81]:
X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv')

In [83]:
y = y['poverty_score_y'].to_numpy()

In [85]:
# forgot about this. for ocd reasons let
X['attended_school'] = X['attended_school'].eq(1)


# Optional. Split into groups.

In [95]:
group1 = X['attended_school'].eq(False)
g1 = X[group1]
school_variates = [
    'highest_school_lvl', 'preschool', 'finish_school_age', 'school_transportation_time', 'school_transportation_cost',
    'school_expenses', 'is_student', 'not_attend_reason1_2', 'not_attend_reason1_3', 'not_attend_reason1_4', 
    'not_attend_reason1_5', 'not_attend_reason1_6', 'not_attend_reason1_7', 'not_attend_reason1_8', 'not_attend_reason1_9', 
    'not_attend_reason1_10', 'not_attend_reason1_11', 'not_attend_reason1_12', 'not_attend_reason1_13', 'not_attend_reason1_14'
]
g1 = g1.drop(columns=school_variates)

# Models

In [7]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:

# Initialize Logistic Regression Model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy

/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.204311152764761

# kNN

In [10]:
knn = KNeighborsClassifier(n_neighbors=40)

# Perform 5-fold cross-validation
cv_scores = cross_val_score(knn, X, y, cv=5, scoring='accuracy')

# Print the accuracy for each fold
print("Cross-validation scores:", cv_scores)
print(f"Average cross-validation accuracy: {cv_scores.mean():.2f}")



Cross-validation scores: [0.17525773 0.13589503 0.10215558 0.13776945 0.19230769]
Average cross-validation accuracy: 0.15


In [11]:
knn.fit(X_train, y_train)
knn.predict_proba(X_test)

array([[0.025, 0.125, 0.15 , ..., 0.05 , 0.   , 0.   ],
       [0.025, 0.05 , 0.15 , ..., 0.05 , 0.   , 0.   ],
       [0.125, 0.075, 0.175, ..., 0.05 , 0.   , 0.   ],
       ...,
       [0.   , 0.075, 0.15 , ..., 0.05 , 0.   , 0.   ],
       [0.025, 0.   , 0.15 , ..., 0.025, 0.   , 0.   ],
       [0.075, 0.15 , 0.225, ..., 0.025, 0.   , 0.   ]])

# Random Forest

In [13]:


# Set up a grid of hyperparameters to search over
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Initialize the Random Forest model
rf = RandomForestClassifier(random_state=42)

# Set up GridSearchCV to search over the hyperparameter grid
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')

# Fit GridSearchCV to the training data
grid_search.fit(X, y)

# Get the best parameters and the best score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation accuracy: {grid_search.best_score_:.2f}")


Best parameters: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 100}
Best cross-validation accuracy: 0.19


In [14]:
# Initialize the Random Forest model
rf = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=5, random_state=42)

# Train the Random Forest model on the training data
rf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf.predict(X_test)

# Evaluate the model using accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy on the test set: {accuracy:.2f}")


Accuracy on the test set: 0.22


In [15]:
# X_test

# Neural network

In [17]:
from sklearn.neural_network import MLPClassifier

# Initialize the MLPClassifier for a neural network
# 'hidden_layer_sizes' defines the number of neurons in the hidden layers (e.g., 100 neurons in one hidden layer)
# 'max_iter' is the maximum number of iterations for training
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42, solver='adam')

# Train the model
mlp.fit(X_train, y_train)

# Make predictions on the test set
y_pred = mlp.predict(X_test)

# Evaluate the model using accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy on the test set: {accuracy:.2f}")


Accuracy on the test set: 0.21


# Do it for reals!

In [19]:
X_test = pd.read_csv('X_TEST.csv')


In [20]:
# Initialize the Random Forest model
rf = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=5, random_state=42)

# Train the Random Forest model on the training data
rf.fit(X, y)

# Make predictions on the test data
y_pred = rf.predict_proba(X_test)

# Evaluate the model using accuracy
# accuracy = accuracy_score(y_test, y_pred)

# print(f"Accuracy on the test set: {accuracy:.2f}")


In [21]:
y_pred

array([[2.53093007e-02, 8.76460896e-02, 2.43833091e-01, ...,
        8.53154367e-03, 2.03286766e-03, 1.04895105e-04],
       [1.19941069e-02, 3.63834406e-02, 1.36402325e-01, ...,
        1.66652578e-02, 2.31381632e-03, 1.61040631e-04],
       [2.07525927e-02, 6.09179380e-02, 2.23392535e-01, ...,
        1.52149537e-02, 1.15617691e-02, 2.72481366e-05],
       ...,
       [4.28394274e-02, 1.04454323e-01, 2.04358148e-01, ...,
        2.88543891e-02, 5.26045401e-03, 3.66889062e-04],
       [4.40887867e-02, 7.32583439e-02, 1.82109793e-01, ...,
        3.40806917e-02, 4.69503833e-03, 1.22897331e-03],
       [4.91515179e-02, 1.19092882e-01, 1.76123305e-01, ...,
        3.10525543e-02, 4.58437054e-03, 7.49941508e-04]])

In [22]:
cols = ['subjective_poverty_1', 'subjective_poverty_2', 'subjective_poverty_3', 'subjective_poverty_4', 
        'subjective_poverty_5', 'subjective_poverty_6',
        'subjective_poverty_7', 'subjective_poverty_8', 'subjective_poverty_9', 'subjective_poverty_10']

y = pd.DataFrame(y_pred, columns=cols)

In [23]:
y

,subjective_poverty_1,subjective_poverty_2,subjective_poverty_3,subjective_poverty_4,subjective_poverty_5,subjective_poverty_6,subjective_poverty_7,subjective_poverty_8,subjective_poverty_9,subjective_poverty_10
0,0.025309,0.087646,0.243833,0.170635,0.114408,0.188915,0.158584,0.008532,0.002033,0.000105
1,0.011994,0.036383,0.136402,0.065581,0.095866,0.462719,0.171914,0.016665,0.002314,0.000161
2,0.020753,0.060918,0.223393,0.094560,0.135650,0.195184,0.242739,0.015215,0.011562,0.000027
3,0.026740,0.027193,0.133244,0.144981,0.121820,0.320215,0.143331,0.074825,0.006752,0.000897
4,0.021059,0.016259,0.100628,0.107433,0.098068,0.421778,0.111222,0.116720,0.004974,0.001858
...,...,...,...,...,...,...,...,...,...,...
1329,0.032418,0.078204,0.192048,0.162700,0.244029,0.160747,0.089845,0.031766,0.007778,0.000465
1330,0.013448,0.031178,0.062911,0.119185,0.174909,0.315854,0.146811,0.114282,0.018759,0.002664
1331,0.042839,0.104454,0.204358,0.210259,0.173608,0.157596,0.072404,0.028854,0.005260,0.000367
1332,0.044089,0.073258,0.182110,0.174890,0.194822,0.191915,0.098911,0.034081,0.004695,0.001229


In [24]:
uids = pd.read_csv('uids.csv')

In [25]:
ans = pd.concat([uids, y], axis=1)


In [26]:
ans.rename(columns={'uid': 'psu_hh_idcode'}, inplace=True)


In [27]:
ans

,psu_hh_idcode,subjective_poverty_1,subjective_poverty_2,subjective_poverty_3,subjective_poverty_4,subjective_poverty_5,subjective_poverty_6,subjective_poverty_7,subjective_poverty_8,subjective_poverty_9,subjective_poverty_10
0,648_6_4,0.025309,0.087646,0.243833,0.170635,0.114408,0.188915,0.158584,0.008532,0.002033,0.000105
1,756_3_3,0.011994,0.036383,0.136402,0.065581,0.095866,0.462719,0.171914,0.016665,0.002314,0.000161
2,164_8_3,0.020753,0.060918,0.223393,0.094560,0.135650,0.195184,0.242739,0.015215,0.011562,0.000027
3,375_4_4,0.026740,0.027193,0.133244,0.144981,0.121820,0.320215,0.143331,0.074825,0.006752,0.000897
4,403_9_1,0.021059,0.016259,0.100628,0.107433,0.098068,0.421778,0.111222,0.116720,0.004974,0.001858
...,...,...,...,...,...,...,...,...,...,...,...
1329,612_1_1,0.032418,0.078204,0.192048,0.162700,0.244029,0.160747,0.089845,0.031766,0.007778,0.000465
1330,738_1_3,0.013448,0.031178,0.062911,0.119185,0.174909,0.315854,0.146811,0.114282,0.018759,0.002664
1331,191_10_2,0.042839,0.104454,0.204358,0.210259,0.173608,0.157596,0.072404,0.028854,0.005260,0.000367
1332,475_9_5,0.044089,0.073258,0.182110,0.174890,0.194822,0.191915,0.098911,0.034081,0.004695,0.001229


In [55]:
ans.to_csv('solution.csv', index=False)
